<a href="https://colab.research.google.com/github/AniketGhorpade/NYC_TaxiTime_Regression/blob/main/NYC_TaxiTime_Prediction_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - 



##### **Project Type**    - Regression (Supervised ML)
##### **Contribution**    - Individual
##### **Team Member 1** - Aniket K. Ghorpade

# **Project Summary -**

Available data in this project was released by New york taxi and Limousine commision. which includes pickup time, geo-coordinates, number of passengers and sevral other variables. Our main task in this project was to build a machine learning model which will predict trip duration time. It was a supervised machine learning problem and we have to predict a contineous variable that's why we have used regression based machine learning algorithms in this project.

Intially we have performed data cleanig and data transformation to bring available data in desirable format. Then we have performed sevral data transformations and derived few new features so that it will be easier to do exploratory analysis. After that we have perfomrmed exploratory data analysis respectively we did univariate analysis, bia-variate analysis to check relationship between different variables followed by multi-variate analysis, so it gave us various insights which are useful to take business decisions. With all above procedure we have finished our exploratory data analysis part of this project.

Then we have performed hypothesis testing to check various hypothetical statements by using various statistical methods. 

After all above procedure we have moved towards machine learning part of our project in which we have made our data model ready by doing various transformations such as label encoding, outlier handlings etc. Then we have selected few machine learning models which are suitable in our case as per the objective of our project. After selecting those 3-4 models we have fitted traning data and tested with our test data. Then did evaluation of those models by using various accuracy metrices and cross validation techniques. Then repeated this process again by doing hyper parameter tunning and this is we are able to find the best supervised regression machine learning model for above problem statment with tunned hyperparameters.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **GitHub Link -**

https://github.com/AniketGhorpade/NYC_TaxiTime_Regression

# **Problem Statement**


**Task is to build a model that predicts the total ride duration of taxi trips in New York City. Apart from that we have to do exploratory data analysis on availble data set which which will be useful to find insights from the dataset.**

# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
## Data Maipulation Libraries
import pandas as pd 
import numpy as np 
from datetime import datetime

## Statistics Library
import scipy.stats as stats

## Data Visualisation Libraray
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import seaborn as sns

## Warnings 
import warnings 
warnings.filterwarnings('ignore')


## Machine Learning 

from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor,RandomForestRegressor 
 
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_squared_error

### Dataset Loading

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Capstone Projects Submission/2. ML Regression - NYC Taxi Time /NYC Taxi Data.csv')

### Dataset First View

In [ ]:
df.head(5)

### Dataset Rows & Columns count

In [ ]:
df.shape

### Dataset Information

In [ ]:
df.info()

#### Duplicate Values

In [ ]:
df.duplicated().value_counts()

No any duplicate value is there in the dataset.

#### Missing Values/Null Values

In [ ]:
df.isna().sum()

Great, No any missing value is there in dataset.

### What did you know about your dataset?

Primary dataset is one released by the NYC Taxi and Limousine Commission, which include pickup time, geo-coordinates, number of passengers, and several other variables.

## ***2. Understanding Your Variables***



1.   Id : A unique identifier for each trip
2.   Vendor_id : A code indicating the provider associated with trip record 
3. pickup_datetime : date and time when the meter was engaged 
4. dropoff_datetime : date and time when the meter was disengaged
5. passenger_count : the number of passengers in the vehicle, driver entered value
6. pickup lattitude : geographical co-ordinates 
7. dropup lattitude : geographical co-ordinates 
8. pickup longitude : geographical co-ordinates 
9. dropup longitude : geographical co-ordinates 
10. store_and_fwd_flag : weather the trip record was held in the vehicle memory,before sending to the server because vehicle did not have a connection to the server.
11. trip_duration : duration of trip in seconds, this is dependant variable




In [ ]:
# Dataset Describe

df.describe(include='all')

### Variables Description 

1. ID, Vendor ID : Represents various ID's of vendors and respective rows, all values in this columns are unique.This are descrete feature we are not going to consider them in analysis as well as in model building.
2. pickup_datetime, dropup_datetime: Almost all values are unique collected from 2016-01-01 to 2016-06-30. 
3. On an avg. 1-2 passengers are trvelling in one car at a time, range of passenger's is from 1 to 9, and median is 1, most of the time only 1 passenger is travelling at a time.
4. Pickup longitude varies in range between -121.93334197998048 to -61.33552932739258.
5. Pickup latitude varies in range between 34.35969543457031 to 51.88108444213867.
6. Dropoff longitude varies in range between -121.9333038330078 to -61.33552932739258.
7. Dropoff latitude varies in range between 32.1811408996582 to 43.92102813720703.
8. store_and_fwd_flag is Off in 99.5 % cases. 
9. Avg. duration of trip is 960 seconds. Minimum is 1 seconds and maximum is 3526282. Median value is 653. There are chances that distribution should be positively skewed.

### Check Unique Values for each variable.

In [ ]:
# Check Unique Values for each variable.
for col in df.columns:
  print(f'{col} : {len(df[col].unique())}')

## 3. ***Data Wrangling***

### Data Wrangling Code

In [ ]:
# Write your code to make your dataset analysis ready.
df.drop(['id'],axis=1,inplace=True)

In [ ]:
df['trip_dur_min'] = round(df['trip_duration']/60,2)

##Calculating distance, by using lattitudes and longitudes

In [ ]:
from math import radians, cos, sin, asin, sqrt
def distance(lat1, lat2, lon1, lon2):
     
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
      
    # calculate the result
    return(c * r)

In [ ]:
df['distance'] = df.apply(lambda row:distance(row['pickup_latitude'],row['dropoff_latitude'],row['pickup_longitude'],row['dropoff_longitude']),axis=1)

In [ ]:
df= df[df['distance']<70]
df= df[df['trip_dur_min']<100]

#####1.Created column day of the week to do analysis on weekday basis.
#####2.Created column speed to do analysis of time that drivers are taking w.r.t distance

In [ ]:
df['pickup_datetime'] = df.apply(lambda row: datetime.strptime(row['pickup_datetime'],'%Y-%m-%d %H:%M:%S'),axis=1)

In [ ]:
df['dropoff_datetime'] = df.apply(lambda row: datetime.strptime(row['dropoff_datetime'],'%Y-%m-%d %H:%M:%S'),axis=1)

In [ ]:
df['pickup_weekday'] = df.apply(lambda row: row['pickup_datetime'].strftime("%A"),axis=1) #Monday=1

In [ ]:
df['pickup_hour'] = df.apply(lambda row: row['pickup_datetime'].hour,axis=1)

In [ ]:
df['speed'] = (df['distance'] * 60) / df['trip_dur_min'] 
df= df[df['speed']<80]

In [ ]:
df.head(3)

### What all manipulations have you done and insights you found?

1. Dropped Id column as we are not going to consider descrete column to do data analysis.
2. Created one column for trip duration in minutes.
3. Calculated distance in kilometers by using pickup,dropoff latitude and longitudes.
4. Converted data types of pickup and dropoff datetime columns into datetime format.
5. Fetched pickup weekday, pickup hour for further analysis from pickup datetime column 


## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

##**I) Analysis for two different vendors**

####**1. Percentage share in rides**

In [ ]:
vendor_per = pd.DataFrame(df.vendor_id.value_counts().reset_index())
vendor_per.rename(columns = {'index':'Vendor ID','vendor_id':'Total Trips'}, inplace = True)

## Pie Chart
plt.figure(figsize=(14, 6))

data = vendor_per['Total Trips']
keys = vendor_per['Vendor ID']
  
# declaring exploding pie
explode = [0.01,0.01]
# define Seaborn color palette to use
palette_color = sns.color_palette('Dark2')
  
# plotting data on chart
plt.pie(data, labels=['Vendor ID 2','Vendor ID 1'], colors=palette_color,explode=explode,autopct='%.0f%%',textprops={'fontsize': 14})


plt.legend(fontsize=14)

plt.title("Share With Respect To Vendor",size=16)
# displaying chart
plt.show() 

1. Pie chart is showing distribution of trips with respect to vendors.
2. There is no much biasness in distribution, both the vendors had trips around 50 % as per data.

####**2. Operation areas of vendor 1 and vender 2 (Operation Range)**

In [ ]:
df_long_p = df.groupby('vendor_id').agg({'pickup_longitude':['min','max']})
df_lat_p = df.groupby('vendor_id').agg({'pickup_latitude':['min','max']})
df_long_d = df.groupby('vendor_id').agg({'dropoff_longitude':['min','max']})
df_lat_d = df.groupby('vendor_id').agg({'dropoff_latitude':['min','max']})

In [ ]:
print(df_long_p)
print(df_lat_p)
print(df_long_d)
print(df_lat_d)

*1) Vendor id 1 : minimum longitude -79.487900, maximum longitude -61.335529, minimum lattitude 34.359695, maximum lattitude 43.911762

2) Vendor id 2 : minimum longitude -121.933342, maximum longitude -73.092278, minimum lattitude 34.359695, maximum lattitude 41.693352*

####**3. Server connection issues with respect to vendors**

In [ ]:
df_flag = df.groupby('vendor_id')
df_flag = pd.DataFrame(df_flag['store_and_fwd_flag'].value_counts())
df_flag.rename(columns = {'store_and_fwd_flag':'store_and_fwd_flag counts'}, inplace = True)
df_flag.reset_index(inplace=True)
df_flag

1) As per above table vendor with Id-1 is facing server connection issues that is also only in 1.5 % cases.

2) Vendor 2 dont have any problem in terms of connection with server there is not even a single incident when vendor 2 has faced any network connection issue.

####**4. Description of trip duration for Vendor 1 and Vendor 2**

In [ ]:
df.groupby('vendor_id')['trip_dur_min'].describe()

1) Above table is showing discription of trip duration in minutes for both the vendors, it is almost equal for both the vendors.

2) Avg. trip duration is 13-14 minutes per trip, other distribution in terms of various quantiles and median is also almost equal.

3) We can conclude from above table that both the vendors are getting trips with almost equal duration.

####**5. Description of distance for Vendor 1 and Vendor 2**

In [ ]:
df.groupby('vendor_id')['distance'].describe()

1) Above table is showing discription of distance in kilometers for both the vendors, it is almost equal for both the vendors same as there trip durations.

2) Avg. trip distance is 3.5 km per trip for both the vendors, other distribution in terms of various quantiles and median is also almost equal.

3) We can conclude from above table that both the vendors are getting trips of equal distances.

##**2.Distribution of contineous variables, Univariate Analysis**

####**I) Distribution of passenger count**

In [ ]:
def distr(DF,x,n_bins=10):

  ''' This function gives univariate distribution of contineous variable in the form of Histogram '''

  legend = ['distribution']

  # Creating histogram
  fig, axs = plt.subplots(1, 1,
                          figsize =(8,5),
                          tight_layout = True)
  
  
  # Remove axes splines
  for s in ['top', 'bottom', 'left', 'right']:
      axs.spines[s].set_visible(False)
  
  # Remove x, y ticks
  axs.xaxis.set_ticks_position('none')
  axs.yaxis.set_ticks_position('none')
    
  # Add padding between axes and labels
  axs.xaxis.set_tick_params(pad = 5)
  axs.yaxis.set_tick_params(pad = 10)
  
  # Add x, y gridlines
  axs.grid(b = True, color ='grey',
          linestyle ='-.', linewidth = 0.5,
          alpha = 0.6)
  kur = round(DF[x].kurt(),2)
  ske = round(DF[x].skew(),2)
  # Add Text watermark
  fig.text(0.9, 0.75, 'Kurtosis: '+str(kur)+' & Skewness: '+str(ske),
          fontsize = 14,
          color ='black',
          ha ='right',
          va ='bottom',
          alpha = 1.0)
  
  # Creating histogram
  N, bins, patches = axs.hist(DF[x], bins = n_bins)
  
  # Setting color
  fracs = ((N**(1 / 5)) / N.max())
  norm = colors.Normalize(fracs.min(), fracs.max())
  
  for thisfrac, thispatch in zip(fracs, patches):
      color = plt.cm.viridis(norm(thisfrac))
      thispatch.set_facecolor(color)
  
  # Adding extra features   
  plt.xlabel(x,size=14)
  plt.ylabel("Count",size=14)
  plt.legend(legend)
  plt.title('Distribution Of '+x,size=16)
  
  # Show plot
  plt.show()

In [ ]:
distr(DF=df,x='passenger_count')
print(f'Avg. Passengers At A Time : {round(df.passenger_count.mean(),2)}')
print(f'Max. Passengers At A Time : {round(df.passenger_count.max(),2)}')

1.With kurtosis 3.43 we could say that there is high peakedness in the data, mean is representing data very well kurtosis is highly positive.
2.With skewness 2.13 it is clearly showing positively skewed distribution, most of the time 1 or 2 passengers are travelling at a time in NYC by using taxi.

####**II) Distribution of distance travelled by customer**

In [ ]:
distr(DF=df,x='distance',n_bins=20)
print(f'Avg. distance of a trip : {round(df.distance.mean(),2)} km')
print(f'Median distance of a trip : {round(df.speed.median(),2)} km')

1.With kurtosis 11.18 we could say that there is high peakedness in the data, mean is representing data very well kurtosis is highly positive, it is leptokurtic distibution.

2.With skewness 2.95 it is clearly showing positively skewed distribution, most of the time 1-10 km is the distance for which customers preffer taxi in NYC.


####**III) Distribution of time taken by driver**

In [ ]:
distr(DF=df,x='trip_dur_min',n_bins=20)
print(f'Avg. time of a trip : {round(df.trip_dur_min.mean(),2)} minutes')
print(f'Median time of a trip : {round(df.trip_dur_min.median(),2)} minutes')

1.With kurtosis 6.87 we could say that there is high peakedness in the data, mean is representing data very well kurtosis is highly positive and distribution is highly leptokurtic.

2.With skewness 2.12 it is clearly showing positively skewed distribution, most of the time 5 to 10 minutes rides has been taken by passengers in NYC by using taxi.


####**IV) Distribution of speed at which drivers are driving**

In [ ]:
distr(DF=df,x='speed',n_bins=20)
print(f'Avg. Speed : {round(df.speed.mean(),2)} km/hr')
print(f'Max. Speed : {round(df.speed.max(),2)} km/hr')

1.With kurtosis 3.23 we could say that there is high peakedness in the data, mean is representing data very well kurtosis is highly positive and distribution is highly leptokurtic.

2.With skewness 1.41 it is clearly showing positively skewed distribution, most of the time drivers are driving at the speed of 14-20 km/hr.

##**3)Traffic w.r.t days of week , time of day**


####**1) Day Time Pickup distribution on hourly basis**

In [ ]:
pickup_hour_dist = pd.DataFrame(df.pickup_hour.value_counts().reset_index())
pickup_hour_dist.rename(columns = {'index':'Pickup Hour','pickup_hour':'TotalTrips'}, inplace = True)
pickup_hour_dist = pickup_hour_dist.sort_values(by='Pickup Hour')
pickup_hour_dist.reset_index(inplace=True)
pickup_hour_dist.drop('index',axis=1,inplace=True)
pickup_hour_dist.head(4)

In [ ]:
plt.figure(figsize=(16, 8))
plots = sns.barplot(x='Pickup Hour', y='TotalTrips', data=pickup_hour_dist)
plots.set_xticklabels(plots.get_xticklabels(), rotation=1, ha="right",size=12)
 
# Iterating over the bars one-by-one
for bar in plots.patches:
    plots.annotate(format(bar.get_height(), '.0f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=8, xytext=(0, 8),
                   textcoords='offset points')
 
# Setting the label for x-axis
plt.xlabel("Pickup Hour", size=14)
 
# Setting the label for y-axis
plt.ylabel("Total Trips", size=14)
 
# Setting the title for the graph
plt.title("Pickup Hours Distribution",size=16)
 
# Finally showing the plot
plt.show()

1. In day time there is very high traffic as compared to the night hours.
2. In office timing entry and exit hours has so much rides to be picked.
3. Graph is clearly showing that pickup traffic is dependant hour pickup hour time, and it is indirectly affecting trip duration also.

####**2) Week day Pickup distribution**

In [ ]:
pickup_weekday_dist = pd.DataFrame(df.pickup_weekday.value_counts().reset_index())
pickup_weekday_dist.rename(columns = {'index':'Pickup Day','pickup_weekday':'TotalTrips'}, inplace = True)
pickup_weekday_dist

In [ ]:
plt.figure(figsize=(14, 6))

sns.set(style="whitegrid", color_codes=True)
plots = sns.barplot(x='Pickup Day', y='TotalTrips', data=pickup_weekday_dist,color='deepskyblue')
plots.set_xticklabels(plots.get_xticklabels(), rotation=1, ha="right",size=12)


 
# Iterating over the bars one-by-one
for bar in plots.patches:
    plots.annotate(format(bar.get_height(), '.0f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=14, xytext=(0, 8),
                   textcoords='offset points')
 
# Setting the label for x-axis
plt.xlabel("Pickup Day", size=14)
 
# Setting the label for y-axis
plt.ylabel("Total Trips", size=14)
 
# Setting the title for the graph
plt.title("Pickup Day Distribution",size=16)
 
# Finally showing the plot
plt.show()

1. On weekend nights that is on fridays and saturday's there is 10 % more traffic as compared to other five working days of week.
2. Almost similar pickups are there on weekdays.

###**4. Number of trips w.r.t time**

In [ ]:
dfda = df[['pickup_datetime']]
dfda['date'] = dfda.apply(lambda row: row['pickup_datetime'].date(),axis=1)

In [ ]:
dfdate = pd.DataFrame(dfda['date'].value_counts().reset_index())
dfdate.rename(columns = {'index':'Date','date':'Total Trips'}, inplace = True)
dfdate.sort_values(by='Date',inplace=True)
dfdate.reset_index(inplace=True)
dfdate.drop('index',axis=1,inplace=True)
dfdate.head(3)

In [ ]:
x = dfdate.Date
y = dfdate['Total Trips']

plt.figure(figsize=(14, 4))


plt.plot(x, y,color='red')
plt.xlabel("Days",size=14)  # add X-axis label
plt.ylabel("Total Trips",size=14)  # add Y-axis label
plt.title("Trips Per Day Time Series",size=16)  # add title
plt.show() 

In [ ]:
df.columns

###**5. Relationship between contineous variables**

#####**1. Relation Between Trip Duration & Distance**

In [ ]:
def Scatter(X,Y):
  '''This function gives scatter plot of two contineous variables'''


  x = df[X]
  y = df[Y]
  colors = df[Y]
  sizes = df[Y]

  plt.figure(figsize = (20, 8))
  plt.ticklabel_format(style = 'plain')
  plt.scatter(x, y, c = colors, s = sizes, alpha = 0.3, cmap = 'PRGn') #viridis
  plt.colorbar()
  
  plt.xlabel(X,size=14)
  plt.ylabel(Y,size=14)
  plt.title(X+" vs "+Y,size=16);

In [ ]:
Scatter(Y='trip_dur_min',X='distance')

#####**2. Relation Between Distance & Speed**

In [ ]:
Scatter(Y='speed',X='distance')

#####**3. Relation Between Trip Duration & Speed**

In [ ]:
Scatter(Y='speed',X='trip_dur_min')

In [ ]:
df.columns

###**6. Is there any relation between Server connection issues with taxi and trip duration or distance**

In [ ]:
dfnint = df[df['store_and_fwd_flag']=='N']
dfint = df[df['store_and_fwd_flag']=='Y']

- **Distribution of trip_duration for trip's which had server connection issue's**

In [ ]:
distr(DF=dfint,x='trip_dur_min',n_bins=20)
print(f'Avg. time of a trip : {round(dfint.trip_dur_min.mean(),2)} minutes')
print(f'Median time of a trip : {round(dfint.trip_dur_min.median(),2)} minutes') 

- **Distribution of trip_duration for trip's which dont had server connection issue's**

In [ ]:
distr(DF=dfnint,x='trip_dur_min',n_bins=20)
print(f'Avg. time of a trip : {round(dfnint.trip_dur_min.mean(),2)} minutes')
print(f'Median time of a trip : {round(dfnint.trip_dur_min.median(),2)} minutes') 

###**7. Correlation Heatmap**

In [ ]:
plt.figure(figsize = (20,10))
colormap = plt.cm.plasma
#sns.heatmap(df[['trip_dur_min','distance','pickup_hour','speed']].corr(),cmap = colormap,annot=True);
sns.heatmap(df.corr(),cmap = colormap,annot=True);
sns.set(font_scale=1.1)

plt.show()

###**8. Pair Plot**

In [ ]:
# plt.figure(figsize = (20,10))
# sns.set(font_scale=0.8)

# sns.pairplot(df, hue ='vendor_id')
# plt.show()

## ***5. Hypothesis Testing***

### Based on your chart experiments, define three hypothetical statements from the dataset. In the next three questions, perform hypothesis testing to obtain final conclusion about the statements through your code and statistical testing.

Hypothesis 1: Passengers which are travelling at a time is changing with change in vendor.

Hypothesis 2: Weekends has more traffic as compared to week days.

Hypothesis 3: Traffic hours on weekdays are different than traffic hours on weekends.

####Hypothesis 1: Passengers which are travelling at a time is changing with change in vendor.

H0 : 'passenger_count' and 'vendor_id' has no relationship.

H1 : 'passenger_count' and 'vendor_id' has relationship.

- As we are analysing two categorical variables, here We are going to use chi-squre test of independance.

In [ ]:
new = pd.crosstab(df.passenger_count,df.vendor_id)
new = pd.DataFrame(new)
new.reset_index(inplace=True)
new['Total'] = new[1] + new[2]
new = new.iloc[1:7,:]
new.set_index('passenger_count',inplace=True)
new.loc['Total'] = new.iloc[:, :].sum()
new

In [ ]:
# significance level
alpha = 0.05

# Calcualtion of Chisquare
chi_square = 0
rows = new.index.unique()
columns = new.columns.unique()
for i in columns:
    for j in rows:
        O = new[i][j]
        E = new[i]['Total'] * new['Total'][j] / new['Total']['Total']
        chi_square += (O-E)**2/E

In [ ]:
# The p-value approach
print("Approach 1: The p-value approach to hypothesis testing in the decision rule")
p_value = 1 - stats.chi2.cdf(chi_square, (len(rows)-1)*(len(columns)-1))
conclusion = "Failed to reject the null hypothesis."
if p_value <= alpha:
    conclusion = "Null Hypothesis is rejected."
        
print("chisquare-score is:", chi_square, " and p value is:", p_value)
print(conclusion)

In [ ]:
# The critical value approach
print("Approach 2: The critical value approach to hypothesis testing in the decision rule")
critical_value = stats.chi2.ppf(1-alpha, (len(rows)-1)*(len(columns)-1))
conclusion = "Failed to reject the null hypothesis."
if chi_square > critical_value:
    conclusion = "Null Hypothesis is rejected."
        
print("chisquare-score is:", chi_square, " and critical value is:", critical_value)
print(conclusion)

**Hypothesis 1 Conclusion: Passenger Count is dependant on Vendor_ID.**

####Hypothesis 2: Weekends has more traffic as compared to week days.

H0 : mean of count of trips on weekday == mean of mean of count of trips on weekend day 

H1 : mean of count of trips on weekday < mean of mean of count of trips on weekend day 

In [ ]:
dfweekday = pd.DataFrame(df.pickup_weekday.value_counts())
dfweekday.reset_index(inplace=True)
dfweekday.rename(columns={'index':'weekday','pickup_weekday':'count'},inplace=True) 

mean_weekday=[]


for i in dfweekday['weekday']:
  if i in ['Saturday','Sunday']:
    mean_weekday.append(2)
  else:
    mean_weekday.append(5)

dfweekday['weekend'] = mean_weekday
dfweekday['mean_'] = round(dfweekday['count']/7,0)
dfweekday.drop('weekend',axis=1,inplace=True)
dfweekday

In [ ]:
weekd = [31866,31485]
weekn = [31167,29968,28913,27846,26730] 
weekd_mean = np.mean(weekd)
weekn_mean = np.mean(weekn)

print(f'mean of week days trips that is m0: {weekn_mean}')
print(f'mean of weekend days trips that is m1: {weekd_mean}')

**Hypothesis 2 Conclusion: Number of trips on weekends are comparatively more than number of trips on weekdays.**

####Hypothesis 3: Traffic hours on weekdays are different than traffic hours on weekends.

In [ ]:
dftraffic = df[['pickup_weekday','pickup_hour']]
weekd = []
for i in dftraffic['pickup_weekday']:
  if i in ['Saturday','Sunday']:
    weekd.append('Weekend')
  else:
    weekd.append('Weekday')
dftraffic['weekd'] = weekd
dftraffic.head(3)

In [ ]:
new = pd.crosstab(dftraffic.pickup_hour,dftraffic.weekd)
new = pd.DataFrame(new)
new.reset_index(inplace=True)
new['Total'] = new['Weekday'] + new['Weekend']
new = new.iloc[0:24,:]
new.set_index('pickup_hour',inplace=True)
new.loc['Total'] = new.iloc[:, :].sum()
new

- As we are analysing two categorical variables, here We are going to use chi-squre test of independance.

In [ ]:
# significance level
alpha = 0.05

# Calcualtion of Chisquare
chi_square = 0
rows = new.index.unique()
columns = new.columns.unique()
for i in columns:
    for j in rows:
        O = new[i][j]
        E = new[i]['Total'] * new['Total'][j] / new['Total']['Total']
        chi_square += (O-E)**2/E

In [ ]:
# The p-value approach
print("Approach 1: The p-value approach to hypothesis testing in the decision rule")
p_value = 1 - stats.chi2.cdf(chi_square, (len(rows)-1)*(len(columns)-1))
conclusion = "Failed to reject the null hypothesis."
if p_value <= alpha:
    conclusion = "Null Hypothesis is rejected."
        
print("chisquare-score is:", chi_square, " and p value is:", p_value)
print(conclusion)

**Hypothesis 3 Conclusion: Traffic hours on weekdays are different than traffic hours on weekends.**

## ***6. Feature Engineering & Data Pre-processing***

### 1. Handling Missing Values

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(inplace=True) #Still confirming it by dropping them 

In [ ]:
print(f'before: {df[df.duplicated()].shape}') # There are around 6 duplicate values in the dataset dropping them 
df.drop_duplicates(keep="first", inplace=True)
print(f'after: {df[df.duplicated()].shape}')

####There is no any missing value present in the dataset.

### 2. Handling Outliers

#####**Let's check below variables for any outliers present in them by, visualising boxplot.**

In [ ]:
def outlier_per(df,x):

  '''this function gives number of outliers present in the feature by using IQR method'''

  q1 = df[x].quantile(0.25)
  q3 = df[x].quantile(0.75)
  iqr = q3-q1
  lw = q1 - (1.5*iqr)
  uw = q3 + (1.5*iqr)

  cnt = 0
  for i in df[x]:
    if (i<lw)| (i>uw):
      cnt+=1

  strng = f'{round(cnt*100/len(df),2)} % Outliers present in column {x} in number it is {cnt}'
  strng2 = f'mean: {round(df[x].mean(),2)}'
  strng3 = f'median: {round(df[x].median(),2)}'
  strng4 = f'minimum: {round(df[x].min(),2)}'
  strng5 = f'maximum: {round(df[x].max(),2)}'
  strng6 = f'kurtosis: {round(df[x].kurt(),2)}'
  strng7 = f'skewness: {round(df[x].skew(),2)}'
  
  return(strng,strng2,strng3,strng4,strng5,strng6,strng7)

In [ ]:
dfcont = ['trip_duration','distance','speed']

In [ ]:
dfoutlier = df[dfcont]
dfoutlier.head(3)

In [ ]:
def outlier_replace(df,x):

  '''this function replaces outliers lower than lower whisker with lower whicker and outliers higher than upper whisker with upper whicker'''

  lw,uw = outlier_detection(df,x)
  lstn = []
  for i in df[x]:
    if (i<lw):
      lstn.append(lw)
    if (i>uw):
      lstn.append(uw)
    else:
      lstn.append(i)

  df[x+'_out_replace'] = lstn

In [ ]:
def outlier_detection(df,x):

  '''this function gives lower whicker and upper whisker of the feature by using IQR method'''

  q1 = df[x].quantile(0.25)
  q3 = df[x].quantile(0.75)
  iqr = q3-q1
  lw = q1 - (1.5*iqr)
  uw = q3 + (1.5*iqr)
  return(lw,uw)

In [ ]:
for i in dfcont:
  outlier_replace(df=dfoutlier,x=i)

In [ ]:
dfoutlier.head(3)

In [ ]:
def boxplot_out(df,X,Y):

    '''this function gives boxplot of variables with and without outliers'''

    ## detecting outliers and calculating there %

    strng0 = f'Boxplot of {X}'
    strng = outlier_per(df,X)[0]
    strng2 = outlier_per(df,X)[1]
    strng3 = outlier_per(df,X)[2]
    strng4 = outlier_per(df,X)[3]
    strng5 = outlier_per(df,X)[4]
    strng6 = outlier_per(df,X)[5]
    strng7 = outlier_per(df,X)[6]
    

    ## plotting figure
    fig, axes = plt.subplots(1,2, figsize=(22,6))
    sns.boxplot(ax=axes[0], data=df,x=X)

    ##labels

    fig.text(0.35, 0.9,strng0,fontsize = 16,color ='black',ha ='right',va ='bottom',alpha = 1.0)
    fig.text(0.45, 0.75,strng,fontsize = 12,color ='black',ha ='right',va ='bottom',alpha = 1.0)
    fig.text(0.45, 0.70,strng2,fontsize = 12,color ='black',ha ='right',va ='bottom',alpha = 1.0)
    fig.text(0.45, 0.65,strng3,fontsize = 12,color ='black',ha ='right',va ='bottom',alpha = 1.0)
    fig.text(0.45, 0.60,strng4,fontsize = 12,color ='black',ha ='right',va ='bottom',alpha = 1.0)
    fig.text(0.45, 0.55,strng5,fontsize = 12,color ='black',ha ='right',va ='bottom',alpha = 1.0)
    fig.text(0.45, 0.50,strng6,fontsize = 12,color ='black',ha ='right',va ='bottom',alpha = 1.0)
    fig.text(0.45, 0.45,strng7,fontsize = 12,color ='black',ha ='right',va ='bottom',alpha = 1.0)

    strng0 = f'Boxplot of {Y}'
    strng = outlier_per(df,Y)[0]
    strng2 = outlier_per(df,Y)[1]
    strng3 = outlier_per(df,Y)[2]
    strng4 = outlier_per(df,Y)[3]
    strng5 = outlier_per(df,Y)[4]
    strng6 = outlier_per(df,Y)[5]
    strng7 = outlier_per(df,Y)[6]
    

    ## plotting figure
    # fig, axes = plt.subplots(1,2, figsize=(22,6))
    sns.boxplot(ax=axes[1], data=df,x=Y)

    ##labels
    # plt.title(f'Boxplot of {Y}',size=14)

    fig.text(0.80, 0.9,strng0,fontsize = 16,color ='black',ha ='right',va ='bottom',alpha = 1.0)
    fig.text(0.85, 0.75,strng,fontsize = 12,color ='black',ha ='right',va ='bottom',alpha = 1.0)
    fig.text(0.85, 0.70,strng2,fontsize = 12,color ='black',ha ='right',va ='bottom',alpha = 1.0)
    fig.text(0.85, 0.65,strng3,fontsize = 12,color ='black',ha ='right',va ='bottom',alpha = 1.0)
    fig.text(0.85, 0.60,strng4,fontsize = 12,color ='black',ha ='right',va ='bottom',alpha = 1.0)
    fig.text(0.85, 0.55,strng5,fontsize = 12,color ='black',ha ='right',va ='bottom',alpha = 1.0)
    fig.text(0.85, 0.50,strng6,fontsize = 12,color ='black',ha ='right',va ='bottom',alpha = 1.0)
    fig.text(0.85, 0.45,strng7,fontsize = 12,color ='black',ha ='right',va ='bottom',alpha = 1.0)
    
    # show plot
    plt.show()

In [ ]:
boxplot_out(df=dfoutlier,X='trip_duration',Y='trip_duration_out_replace')

In [ ]:
boxplot_out(df=dfoutlier,X='distance',Y='distance_out_replace')

### 3. Categorical Encoding

In [ ]:
df.head(3)

####In above dataframe we are going to do categorical encoding for two columns that is 'Store_&_fwd_flag' and 'Pickup_weekday' as store and forward flag has only two variale we will be using Label encoding for it and for pickup weekday columns as there are 7 ordinal catgeorical variables we will be using replace method for this column as from monday to sunday in the range from 1 to 7 as Label encoder will asign numbers to respective days as per it's own way but we actually want to assign monday to sunday in ordinal way from 1 to 7.

In [ ]:
le = LabelEncoder()
df['store_and_fwd_flag'] = le.fit_transform(df['store_and_fwd_flag']) 
df['pickup_weekday'] = df['pickup_weekday'].replace({'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7})
df.head(3)

### 4. Feature Manipulation & Selection

#### 1. Feature Manipulation

In [ ]:
df.columns

1. From above columns we have already created distance column by using pickup and dropoff lattitudes and longitudes.
2. We have created pickup weekday and pickuup hour from date time columns.
3. So we are not going to use above columns as they are parent columns for above two columns which are being created by manupulations.

#### 2. Feature Selection

1. We are selecting features on the basis of information which they are providing and removing features from dataset which are providing same kind of information. For that we have did correlation check.
2. We have performed hypothesis testing to check weather features are relevant or not and from that we have found that pickup weekday and pickup hours are affecting number of trips and traffic times hence they are indirectly affecting trip duration.
3. We have created few features by data manipulations such as distance.

In [ ]:
dfd = df.copy()
dfd = dfd[['vendor_id','passenger_count','store_and_fwd_flag','trip_duration','distance','pickup_weekday','pickup_hour']]
dfd.head(3)

### 5. Data Transformation

#### I don't think there is any need of data transformation as we have already removed outliers from our dataset by replacing them with lower or upper whisker values and most of the columns are of categorical nature and we have already transformed them in desired format.

### 6. Data Scaling

- We are not using satndard scaler to scale our data and bring our data points closer to each other as in our dataset we have only one independant variable which is contineous and all other varaibles are of categorical nature, that's we are not scaling data in this case.
- Another reason to do so is we are not going to implement distance based algorithms in this case, so it is not necesary to bring our data points close to each other.

####**Mulicollenearity check 2 (VIF Method)**
- VIF stands for varaiance inflation factor
- In VIF we regress every independant variable with each other and find the r-square
- After finding r-square we use below VIF formula to find VIF index
VIF index = 1/1-r.square
- If VIF is more than 5 then we say that multi-collinearity exists

In [ ]:
def vif(ind_var):
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    result_df = pd.DataFrame() 
    result_df["Feature"] = ind_var.columns
    result_df["vif"] = [variance_inflation_factor(ind_var.values,i)for i in range(ind_var.shape[1])]
    return result_df

In [ ]:
vif(dfd.drop('trip_duration',axis=1))

- No such multicollinearity is present in independant variables as VIF is less than 5 for almost all variables. So we are not going to perform Priniciple component analysis as our independant variable are only 6 in number and consists around 14,00,000 records so there is no any need of dimensionallity reduction in this case.

### 7. Dimesionality Reduction

#### 1. We have left with only 6 independant features in our dataset after doing feature selection, feature modelling and feature engineering.
#### 2. We have 14,00,000 records stored in 7 columns, hence no any problem will be there due curse of dimentionallity.
#### 3. We have already reduced our features enough, without loosing originality of data.

### 8. Data Splitting

In [ ]:
dfd.shape

1. Let's split data into dependant and independant variables first.

In [ ]:
X = dfd.drop('trip_duration',axis=1)
y = dfd[['trip_duration']]

In [ ]:
from google.colab import files
dfd.to_csv('output.csv', encoding = 'utf-8-sig') 
files.download('output.csv')

2. Now let's split data in the ratio of **80:20** where 80 % will be in training set and 20 % will be in testing set by using train_test_split function available in sklearn library.

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=40)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

3. Now let's split our test data in the ratio of **50:50** where 50 % will be in testing set and 50 % will be in live testing set that is validation set, that we will keep aside for future reference if we will be in the position to test our deployed model on live data.

In [ ]:
X_test,X_live,y_test,y_live = train_test_split(X_test,y_test,test_size=0.5,random_state=40)
print(X_test.shape)
print(X_live.shape)
print(y_test.shape)
print(y_live.shape)

##### We have initially splitted data in 80/20 ratio as training and testing and again we have splitted testing data into two parts 50 % each one as test set and another as a validation set. {Train:Test:Validation = 80:10:10}

## ***7. ML Model Implementation***

##**Model Selection:**


1.   Linear/Lasso/Ridge/Elastic Net: Linear regression has few assumptions; i) Linearity: when it comes to linearity not all the independant features are linearly affecting our dependant feature. ii) Normal Distribution: distribution of data points is not noraml, data is heavily skewed. So basiaclly our data is not following above two important assumptions that's why we will not considering above 4 ML models from implementation point of view.
2.   Distance based algorithms/ Support vector machines: We have huge training dataset which consists around 10,00,000 records so if we use KNN then it will have high runtime as it will scan through all those records at the time of prediction so KNN is not an option in this case, same thing is there for support vectors as it will get so much time for hyperparameter tunning and it will also slow operation with such huge dataset, so that's why we are not considering those algorithms in this case.
3. Decision tress: As we are not dealing with such a problem statment where inpretability/explainibility matters a lot, here we are focusing mostly on accuracy of predictions and we are expecting our working model to be as accarate as possible. So that' why we will consider weak learner based ensamble models or bootstrap aggrigation based random forest alogorithm in this case in this models we will have less explainibilty but we could get better results as compared to pure decision trees.
4. So in this project we will be implementing Adaboost,Gboost,XGboost this three models independantly to check performance sequential ensembling methods and we will also implement Random forest regressor to check performance of bootstrap aggrigation methods, so after seeing results we will consider, any one model out of this four.


####**1. Adaboost Regressor**

In [ ]:
## model training and intial performance

adb = AdaBoostRegressor()
adb.fit(X_train,y_train)
y_pred = adb.predict(X_test)
error = mean_squared_error(y_test,y_pred)
rmse = np.sqrt(error)
er_per = round((rmse*100/np.mean(np.array(y_test))),2)
print(f'{er_per} %')

In [ ]:
## model performance cross validation 


score_adb = cross_val_score(adb,X,y,cv=5)
sc_adb = round(score_adb.mean(),2)
print(f'{sc_adb * 100} %')

In [ ]:
## hyperparameter tunning 

param_adb = {"n_estimators" : [10,30,50,70,90,100,150,200,400],
         "learning_rate" : [0.0001,0.001,0.01,0.1,0.5,1.0]}

tunned_adb=GridSearchCV(AdaBoostRegressor(),param_adb,cv=5)
tunned_adb.fit(X,y)
tunned_adb.best_params_ 

In [ ]:
## tunned model performance cross validation

score_adb = cross_val_score(AdaBoostRegressor(learning_rate= 1.0, n_estimators= 60),X,y,cv=5)
sc_adb = round(score_adb.mean(),2)
print(f'{sc_adb} %')

####**2. Gradient Boosting Regressor**

In [ ]:
## model training and intial performance

gb = GradientBoostingRegressor()
gb.fit(X_train,y_train)
y_pred = gb.predict(X_test)
acc = r2_score(y_test,y_pred)
print(acc)

In [ ]:
## model performance cross validation

score_gb = cross_val_score(gb,X,y,cv=5)
sc_gb = round(score_gb.mean(),2)
print(f'{sc_gb} %')

In [ ]:
## hyperparameter tunning 
param_gb = {"alpha": [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],"learning_rate" : [0.001,0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]}
tunned_gb=GridSearchCV(GradientBoostingRegressor(),param_gb,cv=5)
tunned_gb.fit(X,y)
tunned_gb.best_params_

In [ ]:
## tunned model performance cross validation

score_gb = cross_val_score(GradientBoostingRegressor(alpha= 0.6, learning_rate= 0.3),X,y,cv=5)
sc_gb = round(score_gb.mean(),2)
print(f'{sc_gb} %')

####**3. XG Boost Regressor**

In [ ]:
## model training and intial performance

xgb = XGBRegressor()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
acc = r2_score(y_test,y_pred)
print(acc)

In [ ]:
## model performance cross validation

score_xgb = cross_val_score(xgb,X,y,cv=5)
sc_xgb = round(score_xgb.mean(),2)
print(f'{sc_xgb} %')

In [ ]:
## hyperparameter tunning 
param_xgb =  {'learning_rate' : [0.001,0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],
         'reg_lambda' : [1,2,3,4,5,6,7,8,9,10],
         'max_depth': [1,2,3,4,5,6,7,8,9,10]}
tunned_xgb=GridSearchCV(XGBRegressor(),param_xgb,cv=5)
tunned_xgb.fit(X,y)
tunned_xgb.best_params_

In [ ]:
## tunned model performance cross validation

score_xgb = cross_val_score(XGBRegressor(learning_rate= 0.1, max_depth= 10, reg_lambda= 7),X,y,cv=5)
sc_xgb = round(score_xgb.mean(),2)
print(f'{sc_xgb} %')

####**4. Random Forest Regressor**

In [ ]:
## model training and intial performance

rf = RandomForestRegressor(n_estimators=50)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
acc = r2_score(y_test,y_pred)
print(acc) 

In [ ]:
## model performance cross validation

score_rf = cross_val_score(rf,X,y,cv=5)
sc_rf = round(score_rf.mean(),2)
print(f'{sc_rf} %')

In [ ]:
## hyperparameter tunning 
param_rf =  {"n_estimators":[10,20,30,40,50,60,70,80,90,110,130,150,170,190],'min_impurity_decrease':[0.0,0.5,1],
               "bootstrap":[True,False]}
tunned_rf=GridSearchCV(RandomForestRegressor(),param_rf,cv=5)
tunned_rf.fit(X,y)
tunned_rf.best_params_ 

In [ ]:
## tunned model performance cross validation

score_rf = cross_val_score(RandomForestRegressor(bootstrap= True, min_impurity_decrease= 0.0, n_estimators= 150),X,y,cv=5)
sc_rf = round(score_rf.mean(),2)
print(f'{sc_rf} %')

## ***8.*** ***Future Work (Optional)***

### 1. Save the best performing ml model in a pickle file or joblib file format for deployment process.


In [ ]:
# Save the File

### 2. Again Load the saved model file and try to predict unseen data for a sanity check.


In [ ]:
# Load the File and predict unseen data.

### ***Congrats! Your model is successfully created and ready for deployment on a live server for a real user interaction !!!***

# **Conclusion**

Write the conclusion here.

### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***